In [9]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('msba_only2.xlsx', header = 1)

In [3]:
df.columns = [
    'DateHour',         # Date + hour (YYYYMMDDHH)
    'PagePath',         # Page path + query string
    'UserType',         # New / established
    'Source',           # Session source
    'Sessions',    # Sessions (MSBA Only)
    'EngRate',     # Engagement rate (MSBA Only)
    'KeyEvents',      # Key events (MSBA Only)
    'EngTime',     # Engagement Time (MSBA Only)
    'Sessions1',   # Sessions (Totals)
    'EngRate1',    # Engagement rate (Totals)
    'KeyEvents1',     # Key events (Totals)
    'EngTime1'     # Engagement Time (Totals)
]

df.head()

,DateHour,PagePath,UserType,Source,Sessions,EngRate,KeyEvents,EngTime,Sessions1,EngRate1,KeyEvents1,EngTime1
0,2025093023,/about/centers/morrison-center-for-marketing-a...,established,statics.teams.cdn.office.net,3,1.0,0.0,64668.720944,3,1.0,0.0,64668.720944
1,2025093023,/about/centers/morrison-center-for-marketing-a...,established,statics.teams.cdn.office.net,3,1.0,0.0,72839.788791,3,1.0,0.0,72839.788791
2,2025093023,/about/centers/price-center-for-entrepreneursh...,established,google,3,1.0,0.0,157408.073064,3,1.0,0.0,157408.073064
3,2025093023,/about/office-of-inclusive-excellence/blackand...,established,linkedin.com,3,0.0,0.0,0.000000,3,0.0,0.0,0.000000
4,2025093023,/apply/,established,cn.bing.com,3,1.0,0.0,0.000000,3,1.0,0.0,0.000000


In [4]:
are_identical = df['EngRate'].equals(df['EngRate1'])
print(f"EngRate columns are identical: {are_identical}")

are_identical = df['KeyEvents'].equals(df['KeyEvents1'])
print(f"KeyEvents columns are identical: {are_identical}")

are_identical = df['EngTime'].equals(df['EngTime1'])
print(f"EngTime columns are identical: {are_identical}")

are_identical = df['Sessions'].equals(df['Sessions1'])
print(f"EngTime columns are identical: {are_identical}")

EngRate columns are identical: True
KeyEvents columns are identical: True
EngTime columns are identical: True
EngTime columns are identical: True


In [5]:
df = df.drop(columns = ['EngRate1', 'KeyEvents1', 'EngTime1', 'Sessions1'])

In [6]:
#df.to_csv('cleaned_msba_data.csv', index=False)

In [7]:
import statsmodels.formula.api as smf

# 1. Define Funnel Stages
def get_funnel_stage(path):
    path = str(path).lower()
    if 'apply' in path: return 'Bottom'
    if any(x in path for x in ['career', 'finance', 'admissions']): return 'Middle'
    return 'Top'

df['FunnelStage'] = df['PagePath'].apply(get_funnel_stage)

# 2. Run Regression (Scrapped 'Hour', added 'FunnelStage')
# Interpreting: "How much extra engagement does moving to the next funnel stage drive?"
model_time = smf.ols('EngTime ~ C(FunnelStage) + C(UserType)', data=df).fit()

model_rate = model_rate = smf.ols(
    formula='EngRate ~ C(FunnelStage, Treatment(reference="Top")) * C(UserType)', 
    data=df
).fit()

print(model_time.summary())

print(model_rate.summary())

                            OLS Regression Results                            
Dep. Variable:                EngTime   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     179.0
Date:                Wed, 10 Dec 2025   Prob (F-statistic):          3.90e-153
Time:                        09:47:06   Log-Likelihood:            -1.4319e+06
No. Observations:              100000   AIC:                         2.864e+06
Df Residuals:                   99995   BIC:                         2.864e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [10]:
def categorize_refined(path):
    path = str(path).lower()
    # High Intent (Bottom)
    if 'apply' in path: return 'Application'
    
    # Consideration (Middle)
    if any(x in path for x in ['career', 'placement', 'outcomes']): return 'Career'
    if any(x in path for x in ['financing', 'tuition', 'fellows', 'fees']): return 'Finance'
    if any(x in path for x in ['admissions', 'prerequisites', 'requirements', 'faq']): return 'Admissions'
    if any(x in path for x in ['academics', 'curriculum', 'capstone']): return 'Academics'
    
    # New Categories (Refining "General")
    if 'mba' in path or 'financial-engineering' in path or 'phd' in path: return 'Competitor Program'
    if 'faculty' in path or 'team' in path: return 'Faculty'
    if 'company' in path or 'companies' in path: return 'Corporate'
    if path == '/' or path == '/degrees/master-of-science-in-business-analytics-msba': return 'Homepage'
    
    return 'General Info'

df['Category'] = df['PagePath'].apply(categorize_refined)

# Update Funnel Mapping
def define_funnel_refined(category):
    if category == 'Application': return 'Bottom'
    if category in ['Admissions', 'Finance', 'Career', 'Faculty', 'Corporate']: return 'Middle'
    if category in ['Competitor Program']: return 'Cross-Shopping' # Interesting new segment!
    return 'Top'

df['FunnelStage'] = df['Category'].apply(define_funnel_refined)

# --- FIX 2: HANDLE OUTLIERS (CAPPING) ---
# Cap EngTime at the 99th percentile (approx 30 mins) to remove "sleepers"
cap_value = df['EngTime'].quantile(0.99)
df['EngTime_Capped'] = np.where(df['EngTime'] > cap_value, cap_value, df['EngTime'])

# --- FIX 3: CREATE "ACTIVE" DATASET ---
# For the "Time" regression, only look at non-zero sessions
df_active = df[df['EngTime'] > 0].copy()

In [11]:
model_time2 = smf.ols('EngTime_Capped ~ C(FunnelStage) + C(UserType)', data=df_active).fit()

model_rate2 = model_rate = smf.ols(
    formula='EngRate ~ C(FunnelStage, Treatment(reference="Top")) * C(UserType)', 
    data=df
).fit()

print(model_time2.summary())

print(model_rate2.summary())

                            OLS Regression Results                            
Dep. Variable:         EngTime_Capped   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     228.1
Date:                Wed, 10 Dec 2025   Prob (F-statistic):          8.74e-243
Time:                        09:58:22   Log-Likelihood:            -1.2256e+06
No. Observations:               88075   AIC:                         2.451e+06
Df Residuals:                   88069   BIC:                         2.451e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [17]:
pivot_rate = df_active.pivot_table(
    index='FunnelStage', 
    columns='UserType', 
    values='EngRate', 
    aggfunc='mean'
)

# Metric 2: The "Effort" Metric (High Depth)
# We want to show they are working hard (spending time) when they ARE there.
pivot_time = df_active.pivot_table(
    index='FunnelStage', 
    columns='UserType', 
    values='EngTime_Capped', 
    aggfunc='mean'
)

print("--- QUANTIFYING THE GAP ---")
print("Engagement Rate (Stickiness):")
print(pivot_rate)
print("\nEngagement Time (Effort):")
print(pivot_time)

# Calculate the "Frustration Index" for New Users
# Frustration = High Effort (Time) / Low Success (Rate)
# This is a custom metric you can present to answer "In what way are they underserved?"
frustration_score = pivot_time['new'] / (pivot_rate['new'] * 1000) # Scaled
print("\n--- Frustration Index (New Users) ---")
print(frustration_score)


# ---------------------------------------------------------
# ANALYSIS 2: DIFFERENT FUNNELS FOR DIFFERENT USERS
# Instructor Question: "Are there different funnels?"
# ---------------------------------------------------------

# We need to see the volume flow.
# Do New Users get stuck at the top more than Established users?
pivot_vol = df.pivot_table(
    index='FunnelStage', 
    columns='UserType', 
    values='Sessions', 
    aggfunc='sum'
)

# Calculate % of traffic at each stage (The Funnel Shape)
funnel_shape = pivot_vol.div(pivot_vol.sum(axis=0), axis=1)
print("\n--- Funnel Shape (% of Traffic) ---")
print(funnel_shape)

--- QUANTIFYING THE GAP ---
Engagement Rate (Stickiness):
UserType        (not set)  established       new
FunnelStage                                     
Bottom           0.966292     0.966908  0.955358
Cross-Shopping   0.887417     0.978703  0.977948
Middle           0.939063     0.963278  0.971719
Top              0.924387     0.968673  0.969473

Engagement Time (Effort):
UserType            (not set)    established            new
FunnelStage                                                
Bottom          160597.033439  164252.067207  208922.295233
Cross-Shopping  111798.348669  112001.453292  153971.820933
Middle          151615.989893  168771.698002  211287.429159
Top             132927.312558  125908.903621  166090.408641

--- Frustration Index (New Users) ---
FunnelStage
Bottom            218.684858
Cross-Shopping    157.443794
Middle            217.436692
Top               171.320289
Name: new, dtype: float64

--- Funnel Shape (% of Traffic) ---
UserType        (not set)  esta

In [18]:
df_clean = df[
    (df['FunnelStage'] != 'Cross-Shopping') & 
    (df['UserType'] != '(not set)')
]

# Create the Funnel Shape
pivot = df_clean.pivot_table(
    index='FunnelStage', 
    columns='UserType', 
    values='Sessions', 
    aggfunc='sum'
)

# Sort and Normalize
pivot = pivot.reindex(['Top', 'Middle', 'Bottom'])
funnel_shape = pivot.div(pivot.sum(axis=0), axis=1)

print(funnel_shape.applymap(lambda x: f"{x:.1%}"))

UserType    established    new
FunnelStage                   
Top               43.3%  52.6%
Middle            35.1%  38.7%
Bottom            21.6%   8.7%


/var/folders/n0/d6rp0fkx3blbqwkp9p_0tzbm0000gn/T/ipykernel_41077/1331772084.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(funnel_shape.applymap(lambda x: f"{x:.1%}"))
